In [1]:
!pip install kora -q
from kora import drive
drive.link_nbs()

     |████████████████████████████████| 57 kB 3.3 MB/s 
     |████████████████████████████████| 60 kB 3.2 MB/s 
Mounted at /content/drive


In [6]:
!pwd
%cd /content/drive/MyDrive/ColabsComputer_Vision
import utils.ipynb

/content/drive/MyDrive/ColabsComputer_Vision
/content/drive/MyDrive/ColabsComputer_Vision


ModuleNotFoundError: ignored

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from keras.applications.efficientnet import EfficientNetB0
from keras.models import Model, Sequential
from keras.layers import TimeDistributed, Input, MaxPooling3D, LSTM, Dense, Conv3D, Conv2D, BatchNormalization, Flatten, Dropout, Convolution2D, Activation, MaxPooling2D, GlobalAveragePooling2D
from keras import losses

In [ ]:
""" Global variables"""
NUM_CLASSES = 4
EPOCHS = 30
INSTACK = 16
IMG_SIZE = (224, 224)

In [ ]:
""" Data importing and processing """

In [ ]:
""" Optical flow model """

def build_model(inshape=(16, 224, 224, 3), NUM_CLASSES=4):
  model = Sequential()
  model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2), activation='relu', padding='same'), input_shape=(16, 224, 224, 3)))
  model.add(TimeDistributed(Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
  model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
  
  model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
  model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
  model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
  
  model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
  model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
  model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
  
  model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
  model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
  model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

  model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
  model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
  model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
  
  model.add(TimeDistributed(Flatten()))
  
  model.add(Dropout(0.5))
  model.add(LSTM(256, return_sequences=False, dropout=0.5))
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(NUM_CLASSES, activation='softmax', name='pred'))

  model.summary()
  return model

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_68 (TimeDi  (None, 16, 112, 112, 32)  4736     
 stributed)                                                      
                                                                 
 time_distributed_69 (TimeDi  (None, 16, 110, 110, 32)  9248     
 stributed)                                                      
                                                                 
 time_distributed_70 (TimeDi  (None, 16, 55, 55, 32)   0         
 stributed)                                                      
                                                                 
 time_distributed_71 (TimeDi  (None, 16, 55, 55, 64)   18496     
 stributed)                                                      
                                                                 
 time_distributed_72 (TimeDi  (None, 16, 55, 55, 64)  

In [ ]:
""" Instantiating and compiling model """
of_model = build_model((INSTACK, IMG_SIZE[0], IMG_SIZE[1], 3), NUM_CLASSES)
of_model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=["accuracy"])

